In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import numpy as np
import matplotlib.patches as mpatches
import statsmodels.tsa.statespace as st
import pmdarima as pm
import statsmodels.api as sm

In [64]:
my_data = pd.read_csv("/Users/harleyryan/Desktop/final_data.csv")

In [65]:
# Function to replace minus signs with ones recognizeds by python
def clean(x):
    if isinstance(x, str):
        x = x.replace('−', '-').replace('–', '-').replace('—', '-')
        try:
            return float(x)
        except ValueError:
            return np.nan
    return x

In [66]:
c = ['GDP', 'OWID Education Index', 'Agriculture Workers (%)', 'Industry Workers (%)', 'Service Workers (%)', 'Exports (%)', 'Babies per woman (30 years lagged)', 'Child Mortality Rate (30 years lagged)','Life Expectancy (50 years lagged)']

In [67]:
for col in c:
    my_data[col] = my_data[col].apply(clean)

In [68]:
my_data['GDP'] = my_data["GDP"].astype(float)

In [69]:
my_data.index = my_data['year']

In [70]:
my_data.index = pd.to_datetime(my_data.index, format= '%Y').to_period("Y")

In [71]:
my_data 

,country,year,developed?,GDP,OWID Education Index,Agriculture Workers (%),Industry Workers (%),Service Workers (%),Exports (%),Babies per woman (30 years lagged),Child Mortality Rate (30 years lagged),Life Expectancy (50 years lagged)
year,,,,,,,,,,,,
1991,Afghanistan,1991,no,7.340,10.7,63.40,12.4,24.1,NaN,7.37,351.00,41.2
1992,Afghanistan,1992,no,1.220,10.7,63.70,12.2,24.1,NaN,7.37,345.00,41.4
1993,Afghanistan,1993,no,-5.140,11.3,64.40,11.6,24.0,NaN,7.37,340.00,41.5
1994,Afghanistan,1994,no,-2.570,12.0,64.50,11.5,24.1,NaN,7.36,335.00,41.7
1995,Afghanistan,1995,no,28.300,12.7,64.30,11.5,24.2,NaN,7.35,330.00,41.9
...,...,...,...,...,...,...,...,...,...,...,...,...
2009,Switzerland,2009,yes,-3.750,88.7,3.25,22.1,74.6,58.0,1.52,10.80,71.6
2010,Switzerland,2010,yes,2.500,88.7,3.47,22.4,74.1,64.8,1.55,10.40,71.8
2011,Switzerland,2011,yes,1.340,88.7,3.51,22.3,74.2,66.4,1.54,9.98,71.9


In [72]:
my_data['dummyDeveloped'] = 0
my_data.loc[my_data['developed?']=='yes', ['dummyDeveloped']] =1

In [73]:
my_data['dummy OWID'] = my_data['OWID Education Index']*my_data['dummyDeveloped']
my_data['dummy Mortality'] = my_data['Child Mortality Rate (30 years lagged)']*my_data['dummyDeveloped']

In [74]:
my_data

,country,year,developed?,GDP,OWID Education Index,Agriculture Workers (%),Industry Workers (%),Service Workers (%),Exports (%),Babies per woman (30 years lagged),Child Mortality Rate (30 years lagged),Life Expectancy (50 years lagged),dummyDeveloped,dummy OWID,dummy Mortality
year,,,,,,,,,,,,,,,
1991,Afghanistan,1991,no,7.340,10.7,63.40,12.4,24.1,NaN,7.37,351.00,41.2,0,0.0,0.00
1992,Afghanistan,1992,no,1.220,10.7,63.70,12.2,24.1,NaN,7.37,345.00,41.4,0,0.0,0.00
1993,Afghanistan,1993,no,-5.140,11.3,64.40,11.6,24.0,NaN,7.37,340.00,41.5,0,0.0,0.00
1994,Afghanistan,1994,no,-2.570,12.0,64.50,11.5,24.1,NaN,7.36,335.00,41.7,0,0.0,0.00
1995,Afghanistan,1995,no,28.300,12.7,64.30,11.5,24.2,NaN,7.35,330.00,41.9,0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,Switzerland,2009,yes,-3.750,88.7,3.25,22.1,74.6,58.0,1.52,10.80,71.6,1,88.7,10.80
2010,Switzerland,2010,yes,2.500,88.7,3.47,22.4,74.1,64.8,1.55,10.40,71.8,1,88.7,10.40
2011,Switzerland,2011,yes,1.340,88.7,3.51,22.3,74.2,66.4,1.54,9.98,71.9,1,88.7,9.98


In [75]:
afghan_data = my_data[my_data['country'] == "Afghanistan"]

In [76]:
my_data = my_data.drop(columns=['year', 'country', 'Exports (%)'])

In [77]:
my_data = my_data.sort_index()

In [78]:
my_data =my_data.dropna()

In [79]:
endog =my_data["GDP"]
exog = my_data[['OWID Education Index','dummyDeveloped','dummy OWID']]

In [80]:
model = st.sarimax.SARIMAX(endog, exog = exog, trend= 'c', order=(1,0,1))
results = model.fit(method='powell')
print(results.summary())

Optimization terminated successfully.
         Current function value: 3.006117
         Iterations: 6
         Function evaluations: 419
                               SARIMAX Results                                
Dep. Variable:                    GDP   No. Observations:                  421
Model:               SARIMAX(1, 0, 1)   Log Likelihood               -1265.575
Date:                Sun, 27 Oct 2024   AIC                           2545.150
Time:                        21:23:41   BIC                           2573.449
Sample:                    12-31-1991   HQIC                          2556.334
                         - 12-31-2013                                         
Covariance Type:                  opg                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
intercept                0.1354      0.161      0.84

In [81]:
results.test_heteroskedasticity('breakvar', alternative='two-sided', use_f=True)

array([[1.38297695, 0.05605136]])

In [82]:
exog = my_data['OWID Education Index']

In [83]:
model = st.sarimax.SARIMAX(endog, exog = exog, trend= 'c', order=(1,0,1))
results = model.fit(method='powell')
print(results.summary())

Optimization terminated successfully.
         Current function value: 3.014465
         Iterations: 5
         Function evaluations: 270
                               SARIMAX Results                                
Dep. Variable:                    GDP   No. Observations:                  421
Model:               SARIMAX(1, 0, 1)   Log Likelihood               -1269.090
Date:                Sun, 27 Oct 2024   AIC                           2548.180
Time:                        21:23:41   BIC                           2568.393
Sample:                    12-31-1991   HQIC                          2556.168
                         - 12-31-2013                                         
Covariance Type:                  opg                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
intercept                0.9740      1.034      0.94